# 01. 시퀀스 레이블링(Sequence Labeling)

이번 챕터에서는 파이토치(PyTorch)로 인공 신경망을 이용하여 `태깅 작업`을 하는 모델을 만듭니다. `개체명 인식기`와 `품사 태거`를 만드는데, 이러한 두 작업의 공통점은 RNN의 __다-대-다(Many-to-Many) 작업__ 이면서 또한 앞, 뒤 시점의 입력을 모두 참고하는 __양방향 RNN(Bidirectional RNN)을 사용한다는 점입니다.__

두 개의 실습 챕터를 진행하기 전에 전체적으로 실습이 어떻게 진행되는지 정리해보도록 하겠습니다. 텍스트 분류 개요 챕터와 겹치는 부분에 대해서는 요약하여 설명하므로, 이해가 되지 않는 부분이 있다면 해당 챕터를 참고바랍니다.

## 1. 훈련 데이터에 대한 이해
태깅 작업은 앞서 배운 텍스트 분류 작업과 동일하게 `지도 학습(Supervised Learning)`에 속합니다. 이 챕터에서는 태깅을 해야하는 단어 데이터를 X, 레이블에 해당되는 태깅 정보 데이터는 y라고 이름을 붙였습니다. X에 대한 훈련 데이터는 X_train, 테스트 데이터는 X_test라고 명명하고 y에 대한 훈련 데이터는 y_train, 테스트 데이터는 y_test라고 명명합니다.

이번 챕터에서 X와 y데이터의 쌍(pair)은 병렬 구조를 가진다는 특징을 가집니다. X와 y의 각 샘플의 길이는 같습니다. 예를 들어 품사 태깅 작업을 한다고 가정해보겠습니다. 그리고 X_train와 y_train의 데이터 중 4개의 샘플만 확인해본다고 가정해보겠습니다. 이 때 데이터는 다음과 같은 구조를 가집니다.


|-|	X_train|	y_train	|길이|
|:--:|:--:|:--:|:--:|
|0|	['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb']|	['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O']	|8|
|1|	['peter', 'blackburn']	|['B-PER', 'I-PER']|	2|
|2|	['brussels', '1996-08-22' ]	|['B-LOC', 'O']|	2|
|3|	['The', 'European', 'Commission']|	['O', 'B-ORG', 'I-ORG']|	3|

가령, X_train[3]의 'The'와 y_train[3]의 'O'는 하나의 쌍(pair)입니다. 또한, X_train[3]의 'European'과 y_train[3]의 'B-ORG'는 쌍의 관계를 가지며, X_train[3]의 'Commision'과 y_train[3]의 'I-ORG'는 쌍의 관계를 가집니다.

__이렇게 병렬 관계를 가지는 각 데이터는 정수 인코딩 과정을 거친 후, 모든 데이터의 길이를 동일하게 맞춰주기위한 패딩(Padding) 작업을 거칩니다.__

***
## 2. 시퀀스 레이블링(Sequence Labeling)

__위와 같이 입력 시퀀스(row 1개) X = [x1, x2, x3, ..., xn]에 대하여 레이블 시퀀스 y = [y1, y2, y3, ..., yn]를 각각 부여하는 작업을 시퀀스 레이블링 작업(Sequence Labeling Task)이라고 합니다.__ 태깅 작업은 대표적인 시퀀스 레이블링 작업입니다.

***

## 3. 양방향 RNN(Bidirectional RNN)
```
nn.RNN(input_size = input_size, hidden_size = hidden_size, num_layers = 1, batch_first=True, bidirectional = True)
```
이번 챕터에서도 바닐라 RNN이 아니라 성능이 개선된 RNN인 LSTM이나 GRU 등을 사용합니다. 텍스트 분류 챕터에서는 단방향 RNN을 사용하였지만, 이번 챕터에서는 양방향 RNN을 사용합니다. 이전 시점의 단어 정보 뿐만 아니라, 다음 시점의 단어 정보도 참고하기 위함입니다. __양방향은 기존의 단방향 nn.RNN()에서 bidirectional 인자의 값으로 True를 넣으면 됩니다.__

***
## 4. RNN의 다-대-다(Many-to-Many) 문제
이제 RNN이 어떻게 설계되는지 확인해보도록 하겠습니다. 예를 들어 위에서 설명한 데이터 중 첫번째 데이터에 해당되는 X_train[0]를 가지고 4번의 시점(time steps)까지 RNN을 진행하였을 때의 그림은 다음과 같습니다.

![](https://wikidocs.net/images/page/33805/forwardrnn_ver2.PNG)

하지만 이번 실습에서는 양방향 RNN을 사용할 것이므로 아래의 그림과 같습니다.

![](https://wikidocs.net/images/page/33805/bidirectionalrnn_ver2.PNG)

